In [1]:
import pandas as pd
import requests
import time
import json


In [2]:
# 打算要取得的股票代碼
stock_list_tse = ['0050', '0056', '2330', '2317', '1216']
stock_list_otc = ['6547', '6180']

In [3]:
# 組合API需要的股票清單字串
stock_list1 = '|'.join('tse_{}.tw'.format(stock) for stock in stock_list_tse)      #tse開頭為上市股票。
print(stock_list1)  ###tse_0050.tw|tse_0056.tw|tse_2330.tw|tse_2317.tw|tse_1216.tw

tse_0050.tw|tse_0056.tw|tse_2330.tw|tse_2317.tw|tse_1216.tw


In [4]:
# 6字頭的股票參數不一樣
stock_list2 = '|'.join('otc_{}.tw'.format(stock) for stock in stock_list_otc)     #otc開頭為上櫃股票。
stock_list = stock_list1 + '|' + stock_list2
print(stock_list)  #tse_0050.tw|tse_0056.tw|tse_2330.tw|tse_2317.tw|tse_1216.tw|otc_6547.tw|otc_6180.tw   ## 這個就是把它竄接在一起了。

tse_0050.tw|tse_0056.tw|tse_2330.tw|tse_2317.tw|tse_1216.tw|otc_6547.tw|otc_6180.tw


In [5]:
#　組合完整的URL
query_url = f'http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch={stock_list}'

In [6]:
# 呼叫股票資訊API
response = requests.get(query_url)

In [7]:
# 判斷該API呼叫是否成功
if response.status_code != 200:
  raise Exception('取得股票資訊失敗.')
else:
  print(response.text)

# 將回傳的JSON格式資料轉成Python的dictionary
data = json.loads(response.text)
#print(data)






















{"msgArray":[{"@":"0050.tw","tv":"307","ps":"307","nu":"http://www.yuantaetfs.com/#/RtNav/Index","pid":"9.tse.tw|1","pz":"182.4500","bp":"0","fv":"12","oa":"182.5000","ob":"182.4500","m%":"000000","key":"tse_0050.tw_20240920","^":"20240920","a":"182.4500_182.5000_182.5500_182.6000_182.6500_","b":"182.4000_182.3500_182.3000_182.2500_182.2000_","c":"0050","#":"13.tse.tw|1790","d":"20240920","%":"14:30:00","ch":"0050.tw","tlong":"1726813800000","ot":"14:30:00","f":"102_125_206_116_93_","g":"9_6_26_5_8_","ip":"0","mt":"000000","ov":"15880","h":"184.1000","it":"02","oz":"182.4500","l":"182.4500","n":"元大台灣50","o":"183.1000","p":"0","ex":"tse","s":"307","t":"13:30:00","u":"198.1500","v":"13605","w":"162.1500","nf":"元大台灣卓越50證券投資信託基金","y":"180.1500","z":"182.4500","ts":"0"},{"@":"0056.tw","tv":"137","ps":"137","nu":"http://www.yuantaetfs.com/#/RtNav/Index","pid":"9.tse.tw|6","pz":"38.1500","bp":"0","fv":"24","oa":"38.1600","ob":"38.1500","m%":"000000","key":"tse_0056.tw_202

In [8]:
# 過濾出有用到的欄位
columns = ['c','n','z','tv','v','o','h','l','y', 'tlong']
df = pd.DataFrame(data['msgArray'], columns=columns)
df.columns = ['股票代號','公司簡稱','成交價','成交量','累積成交量','開盤價','最高價','最低價','昨收價', '資料更新時間']

In [9]:
# 紀錄更新時間
def time2str(t):
  print(t)
  t = int(t) / 1000 + 8 * 60 * 60. # UTC時間加8小時為台灣時間

  return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))

# 把API回傳的秒數時間轉成容易閱讀的格式
df['資料更新時間'] = df['資料更新時間'].apply(time2str)

1726813800000
1726813800000
1726813800000
1726813800000
1726813800000
1726813800000
1726813800000


In [10]:
# 顯示股票資訊
display(df)

,股票代號,公司簡稱,成交價,成交量,累積成交量,開盤價,最高價,最低價,昨收價,資料更新時間
0,0050,元大台灣50,182.4500,307,13605,183.1000,184.1000,182.4500,180.1500,2024-09-20 14:30:00
1,0056,元大高股息,38.1500,137,12761,38.2700,38.3900,38.1000,37.9700,2024-09-20 14:30:00
2,2330,台積電,973.0000,26353,59947,981.0000,982.0000,971.0000,960.0000,2024-09-20 14:30:00
3,2317,鴻海,177.0000,24116,81280,180.0000,182.0000,177.0000,176.5000,2024-09-20 14:30:00
4,1216,統一,88.1000,5340,10234,87.8000,88.7000,87.5000,87.8000,2024-09-20 14:30:00
5,6547,高端疫苗,46.3500,92,648,47.5000,47.5000,46.3000,47.0500,2024-09-20 14:30:00
6,6180,橘子,78.7000,29,414,79.3000,79.4000,78.3000,78.4000,2024-09-20 14:30:00
